In [ ]:
from glob import glob
import pandas as pd
from PIL import Image,ImageOps,ImageFile
import os
from tqdm import tqdm
import re
import difflib
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
new_train_path='image_crop'
file_paths='public_train\pill\images\*.jpg'
csv_path='combine_train.csv'

In [ ]:
files=glob(file_paths)
# windows only
files = [f.replace('\\', '/') for f in files]
len(files)

In [ ]:
# if path not exit
if not os.path.exists(new_train_path):
    os.makedirs(new_train_path)

In [ ]:
df = pd.read_json('public_train\pill_pres_map.json')
df.head()

In [ ]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

def convert(txt:str):
    txt = txt.lower()
    txt = remove_accents(txt)
    txt = re.sub(r'^\d+\s*\)\s*', '', txt)
    txt = re.sub(r'\s*sl:.+', '', txt)
    return txt


In [ ]:
files=glob('public_train\prescription\label\*.json')
pres_df = pd.DataFrame()
for file in tqdm(files):
    temp_df = pd.read_json(file)
    # add file name
    temp_df['file_name'] = file.split('\\')[-1]
    pres_df = pd.concat([pres_df,temp_df])
pres_df = pres_df[pres_df['label']=='drugname']
pres_df['mapping'] = pres_df['mapping'].astype(int)
pres_df['text']=pres_df['text'].str.replace(r'^\d+\s*\)\s+','',regex=True).str.lower().apply(lambda x: remove_accents(x))
pres_df.drop(['id'], axis=1, inplace=True)
pres_df.head()

In [ ]:
def is_similar(first, second, ratio):
    return difflib.SequenceMatcher(None, first, second).ratio() > ratio
    
def findid(text,pres_df):
    df = pd.DataFrame([text],columns =['text'])
    result = [s for f in df['text'] for s in pres_df['text'] if is_similar(f,s, 0.9)]
    df = pd.DataFrame(result,columns =['text'])['text'].unique()
    df = pd.DataFrame(df,columns =['text'])
    if not df.empty:
      # print(df.merge(pres_df,on = 'text' )['mapping'].values)
      return df.merge(pres_df,on = 'text' )['mapping'].unique().tolist() # Chỉnh khúc này nếu muốn dùng xác xuất
    else:
      return None

In [ ]:
df = pd.read_json('public_train\pill_pres_map.json')
result = []

image_count = 0
for index, row in tqdm(df.iterrows(), total=len(df)):
    temp_df = pd.read_json('public_train\prescription\label\\'+row['pres'])
    temp_df = temp_df[temp_df['label']=='drugname']
    mapping = temp_df['mapping'].astype(int).values
    vector = [0]*108
    vector[107] = 1
    for i in mapping:
        vector[i] = 1
    temp_df['text']=temp_df['text'].str.replace(r'^\d+\s*\)\s+','',regex=True).str.lower().apply(lambda x: remove_accents(x))
    for text in temp_df.text:
        ids = findid(text,pres_df)
        for id in ids:
            vector[id] = 1
    for pill in row['pill']:
        image_name=pill.replace('json','jpg')
        image_path = f'public_train\pill\image\{image_name}'
        temp_df = pd.read_json(f'public_train\pill\label\{pill}')
        img = Image.open(image_path)
        img = ImageOps.exif_transpose(img)
        for row in temp_df.iloc:
            label = row['label']
        for row in temp_df.iloc:
            x1 = row['x']
            y1 = row['y']
            x2 = x1 + row['w']
            y2 = y1 + row['h']
            label = row['label']
            crop_img = img.crop((x1, y1, x2 , y2))
            save_path = f'{new_train_path}/{image_count}.jpg'
            crop_img.save(save_path)
            image_count+=1
            result.append((save_path, vector,label))

In [ ]:
final_df = pd.DataFrame(result, columns=['pill_path', 'vector','label'])
final_df.head()

In [ ]:
final_df.to_csv(csv_path)